In [23]:
#import lxml.etree as ET
import xml.etree.ElementTree as ET
import regex

In [24]:
tree = ET.parse("C:/Users/user/Dropbox/0_MY498_Dissertation/05_Reference (Codes)/wordvectors/data/mswiki-20210520-pages-articles-multistream.xml")
root = tree.getroot()

In [36]:
print(root.tag)
print(root.attrib)

{http://www.mediawiki.org/xml/export-0.10/}mediawiki
{'{http://www.w3.org/2001/XMLSchema-instance}schemaLocation': 'http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd', 'version': '0.10', '{http://www.w3.org/XML/1998/namespace}lang': 'ms'}


In [41]:
# Iterate over first 10 subelements and their attributes 
for child in root[0:10]:
    print(child.tag, child.attrib)

{http://www.mediawiki.org/xml/export-0.10/}siteinfo {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}
{http://www.mediawiki.org/xml/export-0.10/}page {}


In [42]:
# Get all elements by iterating recursively 
tags = [elem.tag for elem in root.iter()]
tags[:20]

['{http://www.mediawiki.org/xml/export-0.10/}mediawiki',
 '{http://www.mediawiki.org/xml/export-0.10/}siteinfo',
 '{http://www.mediawiki.org/xml/export-0.10/}sitename',
 '{http://www.mediawiki.org/xml/export-0.10/}dbname',
 '{http://www.mediawiki.org/xml/export-0.10/}base',
 '{http://www.mediawiki.org/xml/export-0.10/}generator',
 '{http://www.mediawiki.org/xml/export-0.10/}case',
 '{http://www.mediawiki.org/xml/export-0.10/}namespaces',
 '{http://www.mediawiki.org/xml/export-0.10/}namespace',
 '{http://www.mediawiki.org/xml/export-0.10/}namespace',
 '{http://www.mediawiki.org/xml/export-0.10/}namespace',
 '{http://www.mediawiki.org/xml/export-0.10/}namespace',
 '{http://www.mediawiki.org/xml/export-0.10/}namespace',
 '{http://www.mediawiki.org/xml/export-0.10/}namespace',
 '{http://www.mediawiki.org/xml/export-0.10/}namespace',
 '{http://www.mediawiki.org/xml/export-0.10/}namespace',
 '{http://www.mediawiki.org/xml/export-0.10/}namespace',
 '{http://www.mediawiki.org/xml/export-0.10/}

In [50]:
# Examine the content within a child 
print(ET.tostring(root[1], encoding='utf8').decode('utf8'))

<?xml version='1.0' encoding='utf8'?>
<ns0:page xmlns:ns0="http://www.mediawiki.org/xml/export-0.10/">
    <ns0:title>Laman Utama</ns0:title>
    <ns0:ns>0</ns0:ns>
    <ns0:id>4</ns0:id>
    <ns0:revision>
      <ns0:id>4885379</ns0:id>
      <ns0:parentid>4847632</ns0:parentid>
      <ns0:timestamp>2020-09-05T19:55:52Z</ns0:timestamp>
      <ns0:contributor>
        <ns0:username>Kurniasan</ns0:username>
        <ns0:id>6930</ns0:id>
      </ns0:contributor>
      <ns0:minor />
      <ns0:comment>buang &lt;br&gt; yang wujud disebabkan oleh newline</ns0:comment>
      <ns0:model>wikitext</ns0:model>
      <ns0:format>text/x-wiki</ns0:format>
      <ns0:text bytes="293" xml:space="preserve">&lt;!--                    X x X x X         LAMAN UTAMA         X x X x X                    
--&gt;{{Laman_Utama-Pendahulu}}{{Laman_Utama-Kandungan}}{{Laman_Utama-Akhir}}&lt;!--
                        X x X x X         LAMAN UTAMA         X x X x X                    --&gt;
__NOTOC__ __NOEDITSECT

In [ ]:
for text in root.iter('{http://www.mediawiki.org/xml/export-0.10/}text'):
    print(text)

In [16]:
def clean_text(text):
    global lcode
    
    # Common
    text = regex.sub("(?s)<ref>.+?</ref>", "", text) # remove reference links
    text = regex.sub("(?s)<[^>]+>", "", text) # remove html tags
    text = regex.sub("&[a-z]+;", "", text) # remove html entities
    text = regex.sub("(?s){{.+?}}", "", text) # remove markup tags
    text = regex.sub("(?s){.+?}", "", text) # remove markup tags
    text = regex.sub("(?s)\[\[([^]]+\|)", "", text) # remove link target strings
    text = regex.sub("(?s)\[\[([^]]+\:.+?]])", "", text) # remove media links
    
    text = regex.sub("[']{5}", "", text) # remove italic+bold symbols
    text = regex.sub("[']{3}", "", text) # remove bold symbols
    text = regex.sub("[']{2}", "", text) # remove italic symbols
    
    if lcode in ['ko']: # korean
        text = regex.sub(u"[^ \r\n\p{Hangul}.?!]", " ", text) # Replace unacceptable characters with a space.
    elif lcode in ['ja']: # japanese
        text = regex.sub(u"[^\r\n\p{Han}\p{Hiragana}\p{Katakana}ー。！？]", "", text)
    elif lcode in ['zh']: # chinsese
        text = regex.sub(u"[^\r\n\p{Han}。！？]", "", text)
    elif lcode in ['th']: # thai
        text = regex.sub(u"[^ \r\n\p{Thai}.?!]", " ", text)
    elif lcode in ['ru']: # russian
        text = regex.sub(u"[^ \r\n\p{Cyrillic}.?!\-]", " ", text)
        text = text.lower()
#     elif lcode in ['ar']: # arabic
#         text = regex.sub(u"[^ \r\n\p{Arabic}.?!\-]", " ", text)
    elif lcode in ['hi']: # hindi
        text = regex.sub(u"[^ \r\n\p{Devanagari}.।?!\-]", " ", text)
    elif lcode in ['bn']: # bengali
        text = regex.sub(u"[^ \r\n\p{Bengali}.।?!\-]", " ", text)
    elif lcode in ['de']: # german
        text = regex.sub(u"[^ \r\n\p{Latin}\-'‘’.?!]", " ", text)
    else: # Mostly european languages
        text = regex.sub(u"[^ \r\n\p{Latin}\-'‘’.?!]", " ", text)
        text = text.lower()
    
    # Common
    text = regex.sub("[ ]{2,}", " ", text) # Squeeze spaces.
    return text

In [21]:
fname = 'C:/Users/user/Dropbox/0_MY498_Dissertation/05_Reference (Codes)/wordvectors/data/mswiki-20210520-pages-articles-multistream.xml'
i = 1
ns = "{http://www.mediawiki.org/xml/export-0.10/}"
lcode = 'ms'
for _, elem in ET.iterparse(fname):
    while i <2: 
        print(clean_text(elem.text))
        i += 1

wikipedia
